In [45]:
import pandas as pd

In [48]:
d_work = pd.read_csv("datasets/D_work.csv").drop_duplicates()
d_pens = pd.read_csv("datasets/D_pens.csv").drop_duplicates()
d_clients = pd.read_csv("datasets/D_clients.csv").drop_duplicates()
d_agreement = pd.read_csv("datasets/D_target.csv").drop_duplicates()
d_job = pd.read_csv("datasets/D_job.csv").drop_duplicates()
d_salary = pd.read_csv("datasets/D_salary.csv").drop_duplicates()
d_loan = pd.read_csv("datasets/D_loan.csv").drop_duplicates()
d_close_loan = pd.read_csv("datasets/D_close_loan.csv").drop_duplicates()

In [49]:
d_agreement

,AGREEMENT_RK,ID_CLIENT,TARGET
0,59910150,106804370,0
1,59910230,106804371,0
2,59910525,106804372,0
3,59910803,106804373,0
4,59911781,106804374,0
...,...,...,...
15218,75291424,106819588,0
15219,75291888,106819589,0
15220,75291923,106819590,0
15221,75291960,106819591,0


In [54]:
d_clients

,ID,AGE,GENDER,EDUCATION,MARITAL_STATUS,CHILD_TOTAL,DEPENDANTS,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,FL_PRESENCE_FL,OWN_AUTO
0,106805103,42,1,Среднее,Не состоял в браке,1,0,1,0,Московская область,Московская область,Московская область,1,0
1,106809308,28,1,Среднее специальное,Состою в браке,1,1,1,0,Читинская область,Читинская область,Читинская область,0,0
2,106805867,64,0,Среднее специальное,Состою в браке,2,0,1,1,Иркутская область,Иркутская область,Иркутская область,0,1
3,106808779,54,1,Среднее специальное,Состою в браке,0,0,1,0,Новосибирская область,Новосибирская область,Новосибирская область,1,1
4,106814289,26,0,Среднее специальное,Состою в браке,1,1,1,0,Красноярский край,Красноярский край,Красноярский край,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,106814680,26,1,Среднее,Не состоял в браке,1,1,1,0,Чувашия,Чувашия,Чувашия,0,0
15996,106812464,26,0,Среднее специальное,Состою в браке,0,0,1,0,Карелия,Карелия,Карелия,0,0
15997,106809255,30,1,Среднее специальное,Не состоял в браке,0,0,1,0,Белгородская область,Белгородская область,Белгородская область,0,0
15998,106814593,25,0,Среднее специальное,Состою в браке,0,0,1,0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,0,0


In [81]:
d_closed_loan = d_close_loan[d_close_loan['CLOSED_FL']==1]\
                .reset_index(drop=True)
sum_loan = d_loan.groupby("ID_CLIENT")\
            .agg({"ID_LOAN": pd.Series.nunique})\
            .reset_index()\
            .rename(columns={'ID_LOAN':'LOAN_NUM_TOTAL'})
sum_closed_loans = d_loan\
                    .merge(d_closed_loan, on='ID_LOAN', how='inner')\
                    .groupby("ID_CLIENT")\
                    .agg({"ID_LOAN": pd.Series.nunique})\
                    .reset_index()\
                    .rename(columns={'ID_LOAN':'LOAN_NUM_CLOSED'})

loan_info = sum_loan\
            .merge(sum_closed_loans, on='ID_CLIENT', how='left')\
            .fillna(0)

In [90]:
fin_table = d_agreement\
.merge(d_clients, left_on='ID_CLIENT', 
       right_on='ID',how='left')\
.merge(d_salary, on=['ID_CLIENT'], how='left')\
.merge(loan_info, on=['ID_CLIENT'], how='left')\
[['AGREEMENT_RK', 'ID_CLIENT', 'TARGET', 'AGE', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL',
 'GENDER', 'CHILD_TOTAL', 'DEPENDANTS', 'PERSONAL_INCOME', 'LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED']]

In [91]:
fin_table.isna().sum()

AGREEMENT_RK         0
ID_CLIENT            0
TARGET               0
AGE                  0
SOCSTATUS_WORK_FL    0
SOCSTATUS_PENS_FL    0
GENDER               0
CHILD_TOTAL          0
DEPENDANTS           0
PERSONAL_INCOME      0
LOAN_NUM_TOTAL       0
LOAN_NUM_CLOSED      0
dtype: int64

In [93]:
fin_table.to_csv('datasets/bank_join_report.csv')